In [1764]:
FORM_MAPS = {
    "le génitif": "genitive",
    "gen.": "genitive",
    "gén.": "genitive",
    "gén": "genitive",
    "dat.": "dative",
    "dat": "dative",
    "plur.": "plural",
    "plur": "plural",
    "futur": "future",
    "fut.": "future",
    "participe": "participle",
    "comp.": "comparative",
    "comp": "comparative",
    "compar.": "comparative",
    "compar": "comparative",
    "impér.": "imperative",
    "impér": "imperative",
    "diminutif": "diminutive",
    "vocatif": "vocative",
    "gén. fém.": "genitive_feminine",
    "superl.": "superlative"
}

In [1638]:
import json
with open("sjoestedt_phonetique.json") as inf:
    DATA = json.load(inf)

In [424]:
section = 30

if not section in DATA:
    DATA[section] = []

In [384]:
DATA[section] = []

In [1765]:
import re
REGEX_CASE = rf"^([^(]+) \(([^)]+)\) « ([^»]+) », ({'|'.join(FORM_MAPS.keys())}) ([^(]+) \(([^)]+)\)$"
REGEX_CASE_OF = rf"^([^(]+) \(([^)]+)\),? ({'|'.join(FORM_MAPS.keys())}) de ([^«]+) « ([^»]+) »$"
REGEX_CASE_OF_BOTH = rf"^([^(]+) \(([^)]+)\),? ({'|'.join(FORM_MAPS.keys())}) de ([^«]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_BASE = r"^([^(]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_JUST_FRENCH = r"^([^(]+) « ([^»]+) »$"
REGEX_FROM = r"^([^(]+) \(([^)]+)\) « ([^»]+) »,? de ([^(]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_IN = r"^([^(]+) \(([^)]+)\), dans ([^«]+) « ([^»]+) »$"
REGEX_OR = r"^([^(]+) \(([^)]+)\) ou ([^(]+) \(([^)]+)\) « ([^»]+) »$"

def extract(text, splitter, counter):
    text = text.replace("\u00ad", "").replace("\n", " ")
    for p in text.split(splitter):
            m = re.match(REGEX_BASE, p.strip())
            mjf = re.match(REGEX_JUST_FRENCH, p.strip())
            mf = re.match(REGEX_FROM, p.strip())
            min = re.match(REGEX_IN, p.strip())
            mc = re.match(REGEX_CASE, p.strip())
            mcofb = re.match(REGEX_CASE_OF_BOTH, p.strip())
            mcof = re.match(REGEX_CASE_OF, p.strip())
            mor = re.match(REGEX_OR, p.strip())
            if m:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": m.group(1),
                    "irish": m.group(2),
                    "french": m.group(3),
                })
                counter += 1
            elif mjf:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mjf.group(1),
                    "french": mjf.group(2),
                })
                counter += 1
            elif mf:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mf.group(1),
                    "irish": mf.group(2),
                    "french": mf.group(3).replace("\xad", ""),
                    "from": mf.group(5)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mf.group(4),
                    "irish": mf.group(5),
                    "french": mf.group(6).replace("\xad", ""),
                })
                counter += 1
            elif mor:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mor.group(1),
                    "irish": mor.group(2),
                    "french": mor.group(5),
                    "alt": mor.group(4)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mor.group(3),
                    "irish": mor.group(4),
                    "french": mor.group(5),
                    "alt": mor.group(2)
                })
                counter += 1
            elif min:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": min.group(1),
                    "irish": min.group(2),
                    "in": min.group(3)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": min.group(3),
                    "french": min.group(4),
                })
                counter += 1
            elif mc:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mc.group(1),
                    "irish": mc.group(2),
                    "french": mc.group(3),
                    FORM_MAPS[mc.group(4)]: mc.group(6)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mc.group(5),
                    "irish": mc.group(6),
                    f"{FORM_MAPS[mc.group(4)]}_of": mc.group(2)
                })
                counter += 1
            elif mcofb:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcofb.group(1),
                    "irish": mcofb.group(2),
                    f"{FORM_MAPS[mcofb.group(3)]}_of": mcofb.group(5)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "irish": mcofb.group(5),
                    "transcription": mcofb.group(4),
                    "french": mcofb.group(6),
                    FORM_MAPS[mcofb.group(3)]: mcofb.group(2)
                })
                counter += 1
            elif mcof:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcof.group(1),
                    "irish": mcof.group(2),
                    f"{FORM_MAPS[mcof.group(3)]}_of": mcof.group(4)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcof.group(4),
                    "french": mcof.group(5),
                    FORM_MAPS[mcof.group(3)]: mcof.group(2)
                })
                counter += 1
            else:
                print(p)
    print(counter)


In [1785]:
REST = {
    227: "αχmərtʹ (eachmairt) « rut (chez les chevaux) » ; dɔχmə (dochma) « morose » ; krʹi:ᵊχnəhə (crioch­nuighthe) « terminé » ; ɪ:vʹⁱnʹəs (aoibhneas) « délice » ; ɑχᵊrən (achrann) « diffi­culté » ; sɔχᵊrɩdʹ (sochraid) « enterre­ment » ; fʹi·əvᵊrəs (fiabhras) « fièvre »",
    228: "lɑpᵊrəχɑ̃:n (laprachán) « lutin, kobold » ; ɑ̃mplə (ampladh) « avaler » ; ɛbʹⁱrʹɩ (oibre), gén. de ɔbʷɩrʹ (obair) « travail » ; lɑ:ⁱdʹⁱrʹəχt (láid­reacht) « robus­tesse » ; ì:adᵊrəm (éadrom) « léger » ; ɔkᵊrəs (ocras) « faim » ; pʹi·əkᵊləχ (piaclach) « pénible » ; tɑgᵊrɩmʹ (tagraim) « je mentionne » ; kʷiʃⁱlʹɩ (cuisle) « pouls » ; nə hᴜ·ɛʃⁱlʹɩ (na huaisle) « les gens du monde » ; kʷɩʃlʲɑ:n (caisleán) « château » ; lɑsᵊrəχ (lasrach) « éclair » ; gɑsᵊrə (gasradh) « une bande (de soldats, de jeunes gens) »",
    229: "dʹαrəgnɑ:ⁱrʹɩ (dearg­naire) « grand’honte » ; dʹαrəg (dearg) « rouge » ; nɑ:ⁱrʹɩ (náire) « honte »"
}

In [1792]:
# DATA[section] = []
section = 229
if not section in DATA:
    DATA[section] = []
text = REST[section].replace(" : ", " ; ")
text = text.replace("\u00ad", "")
text = text.replace("»", " »")
text = text.replace("  »", " »")
extract(text, ";", 4)

4


In [1793]:
def split_trans(pos):
    if " ou " in DATA[section][pos]["transcription"]:
        DATA[section][pos]["transcription"] = DATA[section][pos]["transcription"].split(" ou ")
    elif " et " in DATA[section][pos]["transcription"]:
        DATA[section][pos]["transcription"] = DATA[section][pos]["transcription"].split(" et ")
    elif " à côté de " in DATA[section][pos]["transcription"]:
        DATA[section][pos]["transcription"] = DATA[section][pos]["transcription"].split(" à côté de ")

for s in range(len(DATA[section])):
    if "transcription" in DATA[section][s]:
        split_trans(s)

In [1796]:
import json

if str(section) in DATA and section in DATA:
    tmp = DATA[str(section)] + DATA[section]
    DATA[str(section)] = tmp
    del(DATA[section])

with open("sjoestedt_phonetique.json", "w") as outf:
    json.dump(DATA, outf)

In [1795]:
# DATA[section] = DATA[section][0:5]
# del(DATA[section][-1])
# DATA[section][-1]["from"] = "prás"
# DATA[section][13]["in_phrase"] = "ceirt ar gach aon taobh"
# DATA[section][0]["contrast"] = "ᴜmədu꞉ⁱlʹ"
# DATA[section][0]["contrast_id"] = "167_2"
# DATA[section][2]["contrast"] = "go hᴜmərkəχ"
# DATA[section][2]["contrast_id"] = "167_4"
# DATA[section][13]["with_article"] = "ɩnʹ tʲo꞉ləs"
# DATA[section][9]["genitive_of"] = "rᴀɪᵊrk"
# del(DATA[section][10]["french"])
# del(DATA[section][10]["irish"])

# DATA[section][-1]["french_note"] = DATA[section][-1]["french"]
# del(DATA[section][-1]["french"])
# DATA[section] = DATA[section][:8]
# DATA[seqction][-1]["french"] = [DATA[section][-1]["french"], "langue"]
# DATA["6"][31]["french"] = "droit, réclamation"
# DATA[section][4]["dative_of"] = "cearc"
# del(DATA[section][4]["french"])
# DATA[section][3]["dative"] = "circ"

DATA[section][0]["from"] = ["dearg", "náire"]
DATA[section]

[{'section': 229,
  'id': '229_1',
  'transcription': 'dʹαrəgnɑ:ⁱrʹɩ',
  'irish': 'deargnaire',
  'french': 'grand’honte',
  'from': ['dearg', 'náire']},
 {'section': 229,
  'id': '229_2',
  'transcription': 'dʹαrəg',
  'irish': 'dearg',
  'french': 'rouge'},
 {'section': 229,
  'id': '229_3',
  'transcription': 'nɑ:ⁱrʹɩ',
  'irish': 'náire',
  'french': 'honte'}]

In [1502]:
mods = [
    {
      "section": 17,
      "id": "17_23",
      "irish": "lomaim",
      "transcription": "lɔmʷɩmʹ",
      "french": "je me dépouille, je deviens chauve",
      "future": "lomfad"
    },
    {
      "section": 177,
      "id": "177_25",
      "transcription": "sɔnə",
      "irish": "sona",
      "french": "heureux"
    },
    {
      "section": 177,
      "id": "177_23",
      "transcription": "krɔmʷɩmʹ",
      "irish": "cromaim",
      "french": "je courbe"
    },
   {
      "section": 177,
      "id": "177_24",
      "transcription": "fɔnəvər",
      "irish": "fonnmhar",
      "french": "désireux"
    },
]
modwith = ["lo̤mʷɩmʹ", "so̤nə", "kro̤mʷɩmʹ", "fo̤nəvər"]

section = 180
DATA[section] = []
for i in range(len(mods)):
    mod = mods[i]
    mod["section"] = section
    mod["compare_id"] = mod["id"]
    mod["id"] = f"{section}_{i + 1}"
    mod["transcription"] = modwith[i]
    DATA[section].append(mod)
